In [3]:
from __future__ import annotations
import operator
from pathlib import Path
from typing import TypedDict, List, Optional, Literal, Annotated
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_tavily import TavilySearch

In [4]:
load_dotenv()

True

In [5]:
class Task(BaseModel):
    id: int
    title: str
    goal : str = Field(..., description='One sentence describing what the reader should be able to do/understand after this section.')
    bullets: List[str] = Field(...,min_length=3,max_length=5,description='3–5 concrete, non-overlapping subpoints to cover in this section.')
    target_words : int = Field(...,description='Target word count for this section (120–450)')
    section_type: Literal["intro", "core", "examples", "checklist", "common_mistakes", "conclusion"] = Field(
        ...,
        description="Use 'common_mistakes' exactly once in the plan.",
    )

class RouterDecision(BaseModel):
    needs_research:bool
    mode:Literal['open_book','closed_book','hybrid']
    queries:list[str] = Field(default_factory=list)

In [6]:
# Defining the pydantic class
class Plan(BaseModel):
    blog_title: str = Field(description="Title of the blog")
    audience : str = Field(description='Who this blog is for')
    tone : str = Field(description='writing tone (example : crisp,moody,practical ...)')
    tasks: List[Task] = Field(description='breaks the task into sections') 

In [7]:
class blog_writer(TypedDict):
    topic: str

    # routing / research
    mode: str
    needs_research: bool
    queries: List[str]
    evidence: List[EvidenceItem]
    plan: Optional[Plan]

    # workers
    sections: Annotated[List[tuple[int, str]], operator.add]  # (task_id, section_md)
    final: str

In [8]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

In [ ]:
ROUTER_SYSTEM = """You are a routing module for a technical blog planner.

Decide whether web research is needed BEFORE planning.

Modes:
- closed_book (needs_research=false):
  Evergreen topics where correctness does not depend on recent facts (concepts, fundamentals).
- hybrid (needs_research=true):
  Mostly evergreen but needs up-to-date examples/tools/models to be useful.
- open_book (needs_research=true):
  Mostly volatile: weekly roundups, "this week", "latest", rankings, pricing, policy/regulation.

If needs_research=true:
- Output 3–10 high-signal queries.
- Queries should be scoped and specific (avoid generic queries like just "AI" or "LLM").
- If user asked for "last week/this week/latest", reflect that constraint IN THE QUERIES.
"""

def router(state:blog_writer)->dict:
    topic = state['topic']
    decider = llm.with_structured_output(RouterOutput)
    result = decider.invoke([
        SystemMessage(content=ROUTER_SYSTEM),
        HumanMessage(content=f'topic : {topic}')
    ])

    return {
        "needs_research": decision.needs_research,
        "mode": decision.mode,
        "queries": decision.queries,
    }

# Conditional checking 
def route_next(state:blog_writer)->Literal['research','orchestrator']:
  if state['needs_research']:
    return 'research'
  else:
    return 'orchestrator'


In [ ]:
def tavily_search(query:str,max_results:int =5)->List[str]:
    """Searches Tavily for relevant webpages"""
    tool = TavilySearch(max_results=max_results)
    result = tool.invoke({'query':query})
    normalised : List[dict] = []

    for r in result:
        normalised.append({
            'title':r.get('title') or '',
            'url':r.get('url') or ' ',
            'snippet':r.get('content') or r.get('snippet') or ' ',
            'published_at':r.get('published_date') or r.get('published_at'),
            'source':r.get('source'),
        })

    return normalised


RESEARCH_SYSTEM = """You are a research synthesizer for technical writing.

Given raw web search results, produce a deduplicated list of EvidenceItem objects.

Rules:
- Only include items with a non-empty url.
- Prefer relevant + authoritative sources (company blogs, docs, reputable outlets).
- If a published date is explicitly present in the result payload, keep it as YYYY-MM-DD.
  If missing or unclear, set published_at=null. Do NOT guess.
- Keep snippets short.
- Deduplicate by URL.
"""

def research_node(state:blog_writer)->dict:
    


In [10]:
graph = StateGraph(blog_writer)

graph.add_node('router',router)
graph.add_node('research',research)
graph.add_node('orchestrator', orchestrator)
graph.add_node('worker', worker)
graph.add_node('reducer', reducer)

graph.add_edge(START, 'router')
graph.add_conditional_edges("router", route_next, {"research": "research", "orchestrator": "orchestrator"})
graph.add_edge("research", "orchestrator")
graph.add_conditional_edges('orchestrator', fanout, ['worker'])
graph.add_edge('worker', 'reducer')
graph.add_edge('reducer', END)

workflow = graph.compile()

NameError: name 'EvidenceItem' is not defined